In [ ]:
import jax
import jax.numpy as jnp
import numpy as np
from functools import partial
from itertools import combinations, product
import statsmodels.api as sm
import pandas as pd
from scipy.stats import mannwhitneyu

In [2]:
import sys
sys.path.append('/app/python_lib/src')
from robustinfer.drgu import DRGU

# Define Functions

In [3]:
class KeyGen:
    def __init__(self, seed=0):
        self.key = jax.random.PRNGKey(seed)

    def next(self):
        self.key, subkey = jax.random.split(self.key)
        return subkey

rng = KeyGen(42)

In [4]:
def simulation_linear_data(n, ate, error_func):
    eta_true = jnp.array([ 0.0, -0.6])
    w   = jax.random.normal(rng.next(), (n, 1))   # covariates
    Wt  = jnp.concatenate([jnp.ones((n,1)), w], axis=1)
    pi_true    = jax.nn.sigmoid(Wt @ eta_true)                # true propensity
    z          = jax.random.bernoulli(rng.next(), pi_true).astype(jnp.float32)

    # beta_true = jnp.array([0.0, 0.0, 1.0])
    error = error_func(rng.next(), (n,))
    beta_true = jnp.array([ 0.0, ate, 1.0])
    X = jnp.concatenate([jnp.ones((n,1)), z[:,None]*1.0,w], axis=1)
    y = X @ beta_true + error
    return y, z, w, Wt

# Analysis on one simulated data

In [5]:
class ErrorFunc:
    name = ''
    f = None

    def __init__(self, name, f):
            self.name = name
            self.f = f

lognormal_error_func = ErrorFunc('lognormal', lambda key, shape: jax.random.lognormal(key, shape=shape))
y, x, w, _ = simulation_linear_data(n=100, ate=2.0, error_func=lognormal_error_func.f)
    

In [6]:
y.shape, x.shape, w.shape

((100,), (100,), (100, 1))

In [7]:
df = pd.DataFrame({
    "y": y,
    "x": x,
    **{f"w{i+1}": w[:, i] for i in range(w.shape[1])}  # Dynamically create column names for w
})


In [8]:
df.head()

,y,x,w1
0,1.279513,0.0,0.605764
1,3.452183,1.0,0.799044
2,1.433406,1.0,-0.908927
3,1.577025,1.0,-0.635258
4,10.633370,0.0,-1.222659


In [9]:
# Example usage
model = DRGU(df, covariates=["w1"], treatment="x", response="y")
model.fit()

Step 0 gradient norm: 0.7455853819847107
converged after 6 iterations


In [10]:
model.coefficients

Array([ 0.78363335, -0.03352755, -0.965426  ,  2.208195  ,  1.3163865 ,
       -1.1250285 ], dtype=float32)

In [11]:
model.variance_matrix

Array([[ 0.0032916 ,  0.00093761,  0.01224048,  0.00485997, -0.0003902 ,
         0.00617784],
       [ 0.00093761,  0.04584866,  0.01264219,  0.0082063 ,  0.00342252,
        -0.00663543],
       [ 0.01224048,  0.01264219,  0.12225964,  0.041482  ,  0.01941958,
         0.0136685 ],
       [ 0.00485997,  0.00820631,  0.04148201,  0.12721881,  0.09738032,
        -0.04045724],
       [-0.0003902 ,  0.00342252,  0.01941957,  0.09738028,  0.10241216,
        -0.02352003],
       [ 0.00617784, -0.00663543,  0.01366851, -0.04045728, -0.02352004,
         0.12099739]], dtype=float32)

In [12]:
model.summary()

,Names,Coefficient,Null_Hypothesis,Std_Error,Z_Score,P_Value
0,delta,0.783633,0.5,0.057372,4.943722,7.664507e-07
1,beta_0,-0.033528,0.0,0.214123,-0.156581,8.755753e-01
2,beta_1,-0.965426,0.0,0.349656,-2.761070,5.761228e-03
3,gamma_0,2.208195,0.0,0.356677,6.191013,5.977865e-10
4,gamma_1,1.316386,0.0,0.320019,4.113463,3.897669e-05
5,gamma_2,-1.125028,0.0,0.347847,-3.234264,1.219565e-03
